In [3]:
import numpy as np
import pandas as pd

In [26]:
def areas(box):
    return (box[2] - box[0]) * (box[3] - box[1])

def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = areas(boxA)
    boxBArea = areas(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou
box_columns = ['Xmin','Ymin','Xmax','Ymax']

In [34]:
train_data = pd.read_csv('train_data.csv')
train_answers = pd.read_csv('train_answers.csv')
ids = list(set(train_data['itemId']))
users_rating = pd.read_csv("users_rating.csv")
users_rating = users_rating.set_index('userId')

In [6]:
train_full = train_data.merge(train_answers, on=["itemId"])

In [10]:
train_full['iou'] = train_full[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)

In [22]:
train_full.groupby("userId")['iou'].mean().to_csv('users_rating.csv', header=True)

In [44]:
quorum = pd.DataFrame(np.zeros((len(ids), 5), dtype=int), index=ids, columns=box_columns + ['itemId'])
for i in ids:    
    uids = train_data[train_data['itemId'] == i]['userId']
    uid = users_rating.loc[uids].idxmax()['iou']
    ti = train_data.loc[train_data['itemId'] == i].loc[train_data['userId'] == uid][box_columns].max()
    for y in box_columns:
        quorum.loc[i][y] = ti[y]   
    quorum.loc[i]['itemId'] = i

In [45]:
data = quorum.merge(train_answers, on=['itemId'])
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
print(data["iou"].mean())

0.5661568774180677
